### 모듈 임포트하기

In [4]:
import os

import numpy as np
import pandas as pd

from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

### 데이터셋 가져오기

In [3]:
images = []

for dirname, _, filenames in os.walk('./COVID_19_XRAY/train'):
    for filename in filenames:
        if filename == 'labels.csv':
            labels = pd.read_csv('./COVID_19_XRAY/train/labels.csv')
            labels = labels.label.tolist()
            labels = [1 if label=='covid' else 0 for label in labels]
            continue
            
        path = os.path.join(dirname, filename)
        image = Image.open(path)
        image = np.array(image)
        
        images.append(image)

### 데이터셋 분류하기

- train(8):valid(2) 

In [6]:
train_img, valid_img, train_labels, valid_labels = train_test_split(images, labels, test_size=0.2, random_state=33)

### CNN 모델 만들기

In [7]:
class CNN(nn.Module):
    def __init__(self, hid_size, kernel_size, out_size, dropout=0.3):
        super(CNN, self).__init__()
        self.k_size = kernel_size
        self.in_size = 255 - (2 * (self.k_size + 1))
        self.h_size = hid_size
        self.out_size = out_size
        self.dropout = dropout
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=self.k_size, stride=1)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=self.k_size, stride=1)
        self.in_size = (255 - 7) - 7
        self.fc1 = nn.Linear(self.in_size, self.h_size)
        self.fc2 = nn.Linear(self.h_size, self.out_size)
        self.dropout = nn.Dropout(self.dropout)
        
    def forward(self, inputs):
        x = F.relu(self.conv1(inputs))
        x = F.relu(self.conv2(x))
        x = x.view(-1, self.in_size)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x